# Computational Linear Algebra
## A great project on Singular Value Decomposition

### by Paul, Placida, and Sean

## Know your audience
- Third year students with a prior course in linear algebra
- Prior exposure to programming recommended but not required
- In person; we are going to tell ourselves that it's an active learning class
- LMS agnostic
- Access to Syzygy assumed

## Big Ideas and Essential Questions
- Main idea: Singular Value Decomposition is a very stable and fast algorithm with many applications in both math and industry, and it works with **any** matrix
- Core understanding -- the decomposition provides us with lots of useful information:
     - Four fundamental subspaces
     - Rank
     - Norm of a matrix
     - Condition number: how changes in matrix entries can affect solutions
     - Pseudoinverse, least squares
     - Principal component analysis
- Basic understanding: singular values are usually not nice: this is a numerical algorithm
- Essential questions:
     - How to compute it!
     - How to use SVD to obtain all the information listed above

## Learning goals
- How to implement SVD in Python using NumPy
- Relationship between singular values and eigenvalues
- To be able to prove the SVD theorem
- What is so fundamental about the four fundamental subspaces?
- Why is SVD the right way to compute rank?
- How to apply the pseudoinverse to least squares solutions
- Geometric interpretation of singular values

## Learning plan
- Walk through the steps of the SVD algorithm in an example
- Confirm that the results match what the built-in algorithm produces
- Try the algorithm on a few more examples
- Define the four fundamental subspaces, show how to find them from SVD
- Do examples and confirm that SVD gives the same nullspace as "old" methods
- Also use SVD to determine rank in these examples
- Introduce pseudoinverse and have students work through an example

## Notebook design
- Code examples provided
- Ensure students understand how to interpret output by comparing algorithm output to initial worked example
- Students expected to modify/copy code for their own examples